In [4]:
import os

In [5]:
os.chdir('../')

In [6]:
pwd

'c:\\Users\\gagan\\Documents\\projects\\CementStrength'

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class DataValidationConfig:
    source_dir: Path
    good_dir: Path
    bad_dir: Path

In [ ]:
def get_data_validation_config(self) -> DataValidationConfig:
            data_validatoin_config = DataValidationConfig(
                source_dir: 
                good_dir: 
                bad_dir:
                        
                  )
            return data_validation_config

In [8]:
import re
import shutil

In [9]:
from CementStrength.utils import read_yaml,create_directories
from CementStrength.constants import *

In [12]:
config_file = read_yaml(CONFIG_FILE_PATH)

[2023-11-10 12:03:46,563: INFO: common: yaml file: config\config.yml loaded successfully]


In [10]:
from CementStrength import logger

In [22]:
create_directories(config_file.data_validation.good_dir)
create_directories(config_file.data_validation.bad_dir)

# cement_strength_08012020_120021
filepath = config_file.data_validation.source_dir
pattern = r'cement_strength_\d{8}_\d{6}.csv'
for file in os.listdir(filepath):
    match = re.search(pattern, file)
    if match:
        if not os.path.exists(os.path.join(config_file.data_validation.good_dir, file)):
            shutil.copy(os.path.join(config_file.data_validation.source_dir,file), config_file.data_validation.good_dir)
        else:
            logger.info("file already exists")
    else:
        if not os.path.exists(os.path.join(config_file.data_validation.bad_dir, file)):
            shutil.copy(os.path.join(config_file.data_validation.source_dir,file), config_file.data_validation.bad_dir)
        else:
            logger.info("file already exists")    
   

[2023-11-10 12:28:08,075: INFO: common: directory created]
[2023-11-10 12:28:08,077: INFO: common: directory created]
[2023-11-10 12:28:08,079: INFO: 2657052479: file already exists]
[2023-11-10 12:28:08,080: INFO: 2657052479: file already exists]
[2023-11-10 12:28:08,081: INFO: 2657052479: file already exists]


In [14]:
config_file.data_validation.good_dir

'artifacts/data_validation/good'

In [24]:
import pandas as pd

In [25]:
def column_check():
    filepath = config_file.data_validation.good_dir
    for file in os.listdir(filepath):
        with open(file,

In [26]:
filepath = config_file.data_validation.good_dir
for file in os.listdir(filepath):
    print(file)

cement_strength_08012020_120000.csv
cement_strength_08012020_120021.csv


In [30]:

def column_check():
    filepath = config_file.data_validation.good_dir
    for file in os.listdir(filepath):
        print(file)
        df = pd.read_csv(os.path.join(config_file.data_validation.good_dir, file))
        if df.shape[1]!=9:
            shutil.copy(os.path.join(config_file.data_validation.good_dir,file), config_file.data_validation.bad_dir)
            logger.info(f"file {file} transferrred to bad folder")
    logger.info("file check completed")
    

cement_strength_08012020_120000.csv
(1030, 7)
cement_strength_08012020_120021.csv
(1030, 9)


In [31]:
df = pd.read_csv(os.path.join(config_file.data_validation.good_dir, 'cement_strength_08012020_120000.csv'))

In [33]:
df.shape[1]

7

In [3]:
@dataclass(frozen=True)
class DbOperationsConfig:
    source_dir: Path
    db_name: Path
    training_dir: Path
    training_file: Path

In [11]:
class ConfigurationManager:
    
      def __init__(self, 
                config_file= CONFIG_FILE_PATH,
                param_file = PARAMS_FILE_PATH):

            self.config_file = read_yaml(config_file)
            self.param_file = read_yaml(param_file)
            create_directories(self.config_file.data_ingestion.root_dir)  
            create_directories(self.config_file.data_validation.good_dir)
            create_directories(self.config_file.data_validation.bad_dir)
      
      
      
      def get_db_operations_config(self) -> DbOperationsConfig:
            db_operations_config = DbOperationsConfig(
                  source_dir = self.config_file.db_operations.source_dir,
                  db_name= self.config_file.db_operations.db_name,
                  training_dir = self.config_file.db_operations.training_dir,
                  training_file= self.config_file.db_operations.training_file   

                
                        
                  )
            return db_operations_config

In [12]:
from CementStrength.entity import DbOperationsConfig
import pandas as pd
import csv
import sqlite3
import os
from CementStrength import logger
from CementStrength.utils import read_yaml,create_directories


class DbOperations:
    def __init__(self,config: DbOperationsConfig):
        self.config = config
        create_directories(self.config.training_dir)

    def connect_to_db(self):
        logger.info("Connecting to database")
        self.sqliteConnection = sqlite3.connect(self.config.db_name)
        self.cursor = self.sqliteConnection.cursor()
        logger.info("Connection Established")
    
    def close_connection(self):
        self.cursor.close()
        logger.info("Connection Closed")

    def get_data_into_db(self):
        logger.info("Getting data into database")
        dfs = [pd.read_csv(os.path.join(self.config.source_dir,f)) for f in  os.listdir(self.config.source_dir)]
        result = pd.concat(dfs)
        result.to_sql('Cement_Strength',con=self.sqliteConnection,if_exists='replace',index= False)
        self.cursor.close()
        logger.info("Data inserted")
    
    def fetch_data_from_db(self):
        logger.info("fetching data from DB")
        df_sql = pd.read_sql(sql ='select * Cement_Strength;',con = self.sqliteConnection)
        self.cursor.close()
        df_sql.to_csv(self.config.training_file)
        logger.info("data fetched")

In [13]:
from CementStrength.config import ConfigurationManager
from CementStrength.components import DbOperations
class DbOperation_pipeline:
    
    def __init__(self):
        pass

    def main(self): 
      
        db_operation_configurations = ConfigurationManager()
        configs = db_operation_configurations.get_db_operations_config()
        db_operation_obj = DbOperations(config = configs)
        db_operation_obj.connect_to_db()
        db_operation_obj.get_data_into_db()
        db_operation_obj.fetch_data_from_db()

In [15]:
db_operation = DbOperation_pipeline()
db_operation.main()

[2023-11-13 11:54:50,033: INFO: common: yaml file: config\config.yml loaded successfully]
[2023-11-13 11:54:50,041: INFO: common: yaml file: params.yml loaded successfully]
[2023-11-13 11:54:50,042: INFO: common: directory created]
[2023-11-13 11:54:50,043: INFO: common: directory created]


[2023-11-13 11:54:50,044: INFO: common: directory created]


BoxKeyError: "'Box' object has no attribute 'db_operations'"

In [28]:
file = read_yaml("config/config.yml")

[2023-11-13 12:13:12,370: INFO: common: yaml file: config/config.yml loaded successfully]


In [24]:
file.db_operatios

BoxKeyError: "'Box' object has no attribute 'db_operatios'"

In [25]:
file.db_operations.db_name

'db_operatios/database/cement.db'

In [26]:
import sqlite3

In [27]:
sqliteConnection = sqlite3.connect(file.db_operations.db_name)

OperationalError: unable to open database file